In [20]:
import pandas as pd

In [21]:
df = pd.read_csv("../../DATASETS_TRACTATS/TOT_altitud.csv")

In [51]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [52]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "ALTITUD", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df = encoder.fit_transform(df)

In [53]:
df = df[df["CAMPAÑA"] != "22"]

In [54]:
new_df_train = pd.DataFrame()
new_df_test = pd.DataFrame()

In [55]:
from sklearn.model_selection import train_test_split
for c in df["CAMPAÑA"].unique():
    new_df = df[df["CAMPAÑA"] == c]
    train = new_df.sample(frac = 0.8)
    test = new_df.drop(train.index)
    new_df_train = pd.concat([new_df_train, train])
    new_df_test = pd.concat([new_df_test, test])

In [65]:
new_df_train.to_csv("./train_sampled.csv", index=False)
new_df_test.to_csv("./test_sampled.csv", index= False)

In [57]:
new_df_train2 = new_df_train.drop(columns = ["CAMPAÑA"])
new_df_test2 = new_df_test.drop(columns = ["CAMPAÑA"])
X_test = new_df_test2.drop(columns = "PRODUCCION")
y_test = new_df_test2["PRODUCCION"]

In [58]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
def grid_test_year(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [100],
                            "learning_rate": [0.15],
                            "max_depth": [40],
                            "min_child_weight": [1],
                            "gamma":[0.1],
                            "colsample_bytree":[0.4],
                            "booster": ["dart"]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "neg_root_mean_squared_error", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

In [59]:
years_to_test = df["CAMPAÑA"].unique()
model_one_year_out = {}
for year in years_to_test:
    train = new_df_train2[new_df_train["CAMPAÑA"] != year]
    X_train = train.drop(columns = "PRODUCCION")
    y_train = train["PRODUCCION"]
    m_y = grid_test_year(X_train, y_train)
    score_y = m_y.score(X_test, y_test)
    model_one_year_out[year] = (m_y, score_y)

In [68]:
for year in years_to_test:
    print(year)
    print(pd.DataFrame(model_one_year_out[year][0].cv_results_).loc[:,["mean_test_r2", "mean_test_neg_root_mean_squared_error"]])
    print(model_one_year_out[year][1])

14
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.739698                           -6847.747047
-6044.070830724436
15
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.727211                           -6811.558278
-6160.347038328828
16
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.709866                           -6942.065331
-6197.812065047156
17
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.715597                           -7051.683396
-6091.713681512105
18
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.701071                           -6980.239075
-6329.409627561603
19
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.695117                            -7311.44106
-6234.32890869766
20
   mean_test_r2  mean_test_neg_root_mean_squared_error
0         0.721                           -6892.910622
-6154.115672872786
21
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.709614    

In [70]:
new_df_train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
231,14,176,31,12,21,3,2,1,1,0.0000,2394.900,600,70
732,14,572,68,8,46,9,2,1,2,0.0000,3780.000,550,80
813,14,648,72,14,50,4,2,1,1,0.0000,1575.000,455,10
514,14,371,54,8,11,7,2,1,2,0.0000,3980.000,620,20
544,14,398,56,10,35,4,2,1,1,0.0000,1570.000,560,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7731,21,1067,31,9,21,1,1,1,1,1.2068,9197.031,600,70
7710,21,1148,31,12,21,3,2,2,1,1.8800,2981.000,600,70
8119,21,1119,115,3,80,24,1,1,1,1.8125,16890.000,750,0
8141,21,493,63,3,41,5,2,1,1,0.3296,190.060,500,200
